In [1]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [2]:
def get_script(fname:str):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),str(x))
    return 'try{' + f'console.log(`{fname}`,JSON.parse(`{json.dumps(args)}`));'+acc+'}catch(err){console.log(`{fname}`,err);}'
  return replacer
def script_runner(driver):
  def run_script(name:str,verbose=False):
    def script(*args:Any):
      script_string = get_script(name)(*args)
      answer = driver.execute_script(script_string)
      if verbose:
        print('script_string: ',script_string,' Answer: ', answer)
      return answer
    return script
  return run_script

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed

In [4]:
driver = GetBrowser()

In [5]:
run_script = script_runner(driver)

In [6]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [7]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)

In [8]:
class RedditPostFeed(Feed):
	def __init__(self, driver:webdriver.Chrome ,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Post Feed',driver=driver,handle=handle)
		# Store the WebDriver instance
		self.verbose = verbose
		self.focus()
		run_script('listen to clicks')()
		run_script('post/catch first comment',verbose=self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not run_script('check cursor',verbose=self.verbose)():
				continue
			run_script('scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if run_script('check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield run_script('post/parse comment',verbose=self.verbose)()
			run_script('cursor next')()

class RedditFeed(Feed):
	def __init__(self, driver:webdriver.Chrome,skip=True,handle:str|None=None,verbose=False):
		super().__init__(name='Reddit Feed',handle=handle,driver=driver)
		self.verbose = verbose
		if not skip:
			self.driver.get('https://www.reddit.com/')
			run_script('listen to clicks')()
		run_script('init_reddit',verbose=self.verbose)()
	

	def __iter__(self) -> Iterator[Any]:
		while True:
			self.focus()
			if not run_script('check cursor',verbose=self.verbose)():
				continue
			run_script('scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			for i in range(10):
				if run_script('check still scrolling')():
					time.sleep(0.1)
				else:
					break
				if i == 9:
					raise Exception('Not finished scrolling')
			yield run_script('parse post',verbose=self.verbose)()
			run_script('cursor next')()

	def click_post(self, newTab=True):
		for i in range(10):
			self.focus()
			if i == 9:
				raise Exception('cursor not present')
			if not run_script('check cursor',verbose=self.verbose)():
				continue
			else:
				break
  
		id = uuid()
		run_script('mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		h3 = element.find_element(By.CSS_SELECTOR,'h3')
		element = element if h3 is None else h3
		actions = ActionChains(driver)
		actions.key_down(Keys.CONTROL).click(element).key_up(Keys.CONTROL).perform()
		# run_script('catch first comment')()
		return RedditPostFeed(self.driver,self.driver.window_handles[-1],verbose=self.verbose)

	def close(self):
		self.focus()
		# Close the WebDriver instance
		self.driver.quit()

In [9]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [10]:
# for x in islice(rf,15):
#   print(x)
#   time.sleep(2*max(1,len(x) * 0.01))

In [11]:
e = rf.click_post()

In [12]:
driver.current_window_handle

'010C60ECB260617618817A53B61A7EAB'

In [13]:
driver.switch_to.window(driver.window_handles[2])

IndexError: list index out of range

In [ ]:
run_script('cluster')()

In [18]:
d = run_script('post/parse comment')()

In [20]:
d

{'commentText': "Si tu contestes -> Tu travailles pas, donc tu dois te taire\n\nSi tu contestes pas -> T'es d'accord avec le gouvernement\n\nMerveilleux.",
 'depth': 0,
 'replyClass': 'rfecKUUgOnjiqmWcPKRO',
 'up_votes': '483',
 'userName': 'Ghaenor',
 'userUrl': 'https://www.reddit.com/user/Ghaenor/'}

In [ ]:
# de-clutter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })